In [1]:
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import unicodedata
import re
import numpy as np
import os
import io

In [2]:
# path_to_zip = keras.utils.get_file('spa-eng.zip', origin='https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip', extract=True)

In [3]:
# path_to_zip

In [4]:
# path_to_file = os.path.join(os.path.dirname(path_to_zip), 'spa-eng', 'spa.txt')
# path_to_file

In [5]:
path_to_file = os.path.join('spa-eng', 'spa.txt')

In [6]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

In [7]:
def preprocess_senetence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.~,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.~,]+", " ", w)
    w = w.rstrip().strip()
    w = '<start> ' + w + ' <end>'
    return w

In [8]:
en_sentence = "I have some friends to help."
preprocess_senetence(en_sentence)

'<start> i have some friends to help . <end>'

In [9]:
sp_sentence = "Aún no he dicho nada."
preprocess_senetence(sp_sentence)

'<start> aun no he dicho nada . <end>'

In [10]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    word_pairs = [[preprocess_senetence(w) for w in l.split('\t')] for l in lines[:num_examples]]
    return zip(*word_pairs)

In [11]:
print(create_dataset(path_to_file, 2))

In [12]:
en, sp = create_dataset(path_to_file, None)

In [13]:
len(en)

118964

In [14]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [15]:
def tokenize(lang):
    lang_tokenizer = keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

In [16]:
def load_dataset(path, num_examples=None):
    targ_lang, inp_lang = create_dataset(path, num_examples)
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, target_lang_tokenizer = tokenize(targ_lang)
    return input_tensor, target_tensor, inp_lang_tokenizer, target_lang_tokenizer

In [17]:
input_tensor, target_tensor, input_lang_tokenizer, target_lang_tokenizer = load_dataset(path_to_file, 20000)

In [18]:
input_tensor

array([[  1,  97,   3, ...,   0,   0,   0],
       [  1, 179,   3, ...,   0,   0,   0],
       [  1, 466,   3, ...,   0,   0,   0],
       ...,
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0],
       [  1,   7,  10, ...,   0,   0,   0]])

In [19]:
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(input_tensor, target_tensor, test_size=0.2)

In [21]:
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print(t, ' .... ', lang.index_word[t])

In [22]:
convert(input_lang_tokenizer, input_tensor[0])

1  ....  <start>
97  ....  ve
3  ....  .
2  ....  <end>


In [23]:
input_tensor[0]

array([ 1, 97,  3,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [24]:
target_lang_tokenizer.word_index

{'<start>': 1,
 '<end>': 2,
 '.': 3,
 'i': 4,
 'you': 5,
 '?': 6,
 'tom': 7,
 'is': 8,
 'it': 9,
 's': 10,
 'a': 11,
 'he': 12,
 't': 13,
 'the': 14,
 'we': 15,
 'm': 16,
 'me': 17,
 're': 18,
 'that': 19,
 'this': 20,
 'to': 21,
 'do': 22,
 'are': 23,
 'can': 24,
 'my': 25,
 'they': 26,
 'was': 27,
 'she': 28,
 'don': 29,
 'have': 30,
 'your': 31,
 'go': 32,
 'what': 33,
 'in': 34,
 'not': 35,
 'll': 36,
 'like': 37,
 'here': 38,
 'on': 39,
 'him': 40,
 'let': 41,
 'be': 42,
 'did': 43,
 'know': 44,
 'come': 45,
 'up': 46,
 'am': 47,
 'want': 48,
 'how': 49,
 ',': 50,
 'mary': 51,
 'get': 52,
 'who': 53,
 'very': 54,
 'now': 55,
 'need': 56,
 'has': 57,
 'please': 58,
 'no': 59,
 'there': 60,
 'help': 61,
 'her': 62,
 'love': 63,
 'at': 64,
 'see': 65,
 'just': 66,
 'out': 67,
 'his': 68,
 've': 69,
 'got': 70,
 'for': 71,
 'where': 72,
 'look': 73,
 'stop': 74,
 'one': 75,
 'us': 76,
 'good': 77,
 'car': 78,
 'too': 79,
 'so': 80,
 'all': 81,
 'why': 82,
 'will': 83,
 'an': 84,
 'hom

In [25]:
BUFFER_SIZE = len(X_train)
BATCH_SIZE = 64
steps_per_epoch = len(X_train) // BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(input_lang_tokenizer.word_index) + 1
vocab_targ_size = len(target_lang_tokenizer.word_index) + 1

In [26]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(BUFFER_SIZE)

In [27]:
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [28]:
class Encoder(keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True)
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state
    def initilize_hidden_state(self):
        return tf.zeros((self.batch_size, self.enc_units))

In [29]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

In [30]:
encoder

In [31]:
simple_hidden = encoder.initilize_hidden_state()
simple_hidden

<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [32]:
example_input_batch, example_target_batch = next(iter(dataset))

In [33]:
encoder(example_input_batch, simple_hidden)

(<tf.Tensor: shape=(64, 16, 1024), dtype=float32, numpy=
 array([[[-0.01097779,  0.00128966, -0.00544116, ...,  0.00261882,
           0.00268693, -0.00259845],
         [-0.01372856,  0.002827  , -0.00930369, ..., -0.00146007,
           0.00274714, -0.00179466],
         [-0.00739557, -0.00508326, -0.00965582, ..., -0.00308934,
           0.0116131 , -0.00600911],
         ...,
         [ 0.00471458,  0.01859427, -0.01342205, ..., -0.00354091,
          -0.00568658,  0.00417595],
         [ 0.00475038,  0.01862186, -0.01346868, ..., -0.00349747,
          -0.00567781,  0.00415488],
         [ 0.00477609,  0.01863538, -0.01349867, ..., -0.00346668,
          -0.0056722 ,  0.00413909]],
 
        [[-0.01097779,  0.00128966, -0.00544116, ...,  0.00261882,
           0.00268693, -0.00259845],
         [-0.01466605,  0.0008149 , -0.00633856, ..., -0.00260375,
          -0.00759092, -0.00171117],
         [ 0.00014679, -0.00453507,  0.00187358, ...,  0.00178376,
          -0.00903995,  0.0

In [34]:
simple_output, simple_states = encoder(example_input_batch, simple_hidden)

In [35]:
class Attention(keras.layers.Layer):
    def __init__ (self, units):
        super(Attention, self).__init__()
        self.W1 = keras.layers.Dense(units)
        self.W2 = keras.layers.Dense(units)
        self.V = keras.layers.Dense(1)
    def call (self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(hidden_with_time_axis)))
        atteion_weights = tf.nn.softmax(score, axis=1)
        context_vector = atteion_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, atteion_weights

In [36]:
attention_layer = Attention(10)
attention_layer(simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 3.8181170e-04,  1.2385931e-02, -1.1194648e-02, ...,
         -2.6743428e-03, -3.1128491e-03,  1.8179978e-03],
        [ 1.3131602e-03,  9.8076034e-03, -8.4452266e-03, ...,
         -2.4205579e-03, -3.1172873e-03,  2.2503319e-03],
        [ 8.6637912e-05,  9.0624159e-03, -8.1556756e-03, ...,
         -2.4725369e-03, -2.3503955e-03,  6.0557772e-04],
        ...,
        [ 3.6556926e-04,  8.9787636e-03, -8.6469455e-03, ...,
         -1.7828703e-03, -4.6531036e-03,  2.1244679e-03],
        [ 2.6635730e-03,  8.5681342e-03, -9.5077734e-03, ...,
         -8.0367189e-04, -2.3470193e-03,  2.9752818e-03],
        [-5.2056811e-04,  7.1796561e-03, -8.9662727e-03, ...,
         -3.0751736e-03, -3.2451467e-03, -1.6032433e-04]], dtype=float32)>,
 <tf.Tensor: shape=(64, 16, 1), dtype=float32, numpy=
 array([[[0.06183003],
         [0.0621557 ],
         [0.06190022],
         ...,
         [0.06310068],
         [0.06311072],
         [0.0

In [37]:
attention_result, attention_weights = attention_layer(simple_hidden, simple_output)

In [38]:
class Decoder(keras.Model):
    def __init__ (self, vocab_size, embedding_dim, dec_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True)
        self.fc = keras.layers.Dense(vocab_size)
        self.attention = Attention(self.dec_units)
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [39]:
decoder = Decoder(vocab_targ_size, embedding_dim, units, BATCH_SIZE)

In [40]:
decoder(tf.random.uniform((BATCH_SIZE, 1)), simple_hidden, simple_output)

(<tf.Tensor: shape=(64, 3727), dtype=float32, numpy=
 array([[ 0.00405281, -0.0020623 , -0.00224488, ..., -0.00200103,
         -0.00190322,  0.00233592],
        [ 0.0037182 , -0.00198614, -0.00168528, ..., -0.0025641 ,
         -0.0019875 ,  0.00316469],
        [ 0.00376004, -0.0021118 , -0.00156196, ..., -0.00271897,
         -0.00153099,  0.00273313],
        ...,
        [ 0.00366892, -0.00180054, -0.00194933, ..., -0.0022993 ,
         -0.00180112,  0.00290346],
        [ 0.00408969, -0.00208865, -0.00186774, ..., -0.00216026,
         -0.00150029,  0.00240201],
        [ 0.00377634, -0.00224088, -0.00205109, ..., -0.00222273,
         -0.00081091,  0.00374181]], dtype=float32)>,
 <tf.Tensor: shape=(64, 1024), dtype=float32, numpy=
 array([[ 1.13477930e-02, -3.03088920e-03,  1.89501618e-04, ...,
         -1.53357326e-03, -3.99182830e-03, -2.34363150e-04],
        [ 1.09608006e-02, -2.65010400e-03, -3.97878794e-05, ...,
         -1.41356327e-03, -2.89602135e-03, -4.86949080e-04],

In [41]:
optimizer = keras.optimizers.Adam()
loss_object = keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [42]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

In [43]:
checkpoint_dir = 'chckpnts'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

In [46]:
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [47]:
EPOCH = 10
for epoch in range(EPOCH):
    enc_hidden = encoder.initilize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        print('Epoch: ', epoch)
        print('Loss: ', batch_loss.numpy())
    checkpoint.save(file_prefix='test1')

Epoch:  0
Loss:  4.5737476
Epoch:  0
Loss:  4.6943135
Epoch:  0
Loss:  4.5108423
Epoch:  0
Loss:  4.25315
Epoch:  0
Loss:  3.4254947
Epoch:  0
Loss:  4.317342
Epoch:  0
Loss:  3.8291924
Epoch:  0
Loss:  3.2174811
Epoch:  0
Loss:  2.873603
Epoch:  0
Loss:  2.8738282
Epoch:  0
Loss:  3.0235312
Epoch:  0
Loss:  2.9348104
Epoch:  0
Loss:  3.147251
Epoch:  0
Loss:  2.989672
Epoch:  0
Loss:  3.0475347
Epoch:  0
Loss:  2.8246558
Epoch:  0
Loss:  2.8065877
Epoch:  0
Loss:  2.8114467
Epoch:  0
Loss:  2.8227468
Epoch:  0
Loss:  2.8668027
Epoch:  0
Loss:  2.816814
Epoch:  0
Loss:  2.7051346
Epoch:  0
Loss:  2.6131692
Epoch:  0
Loss:  2.7440991
Epoch:  0
Loss:  2.6966026
Epoch:  0
Loss:  2.639869
Epoch:  0
Loss:  2.588343
Epoch:  0
Loss:  2.5880847
Epoch:  0
Loss:  2.6288202
Epoch:  0
Loss:  2.5454037
Epoch:  0
Loss:  2.5403984
Epoch:  0
Loss:  2.6415277
Epoch:  0
Loss:  2.4921331
Epoch:  0
Loss:  2.4782462
Epoch:  0
Loss:  2.5794322
Epoch:  0
Loss:  2.5813248
Epoch:  0
Loss:  2.506498
Epoch:  0
L

In [48]:
def evaluate(sentence):
    sentence = preprocess_senetence(sentence)
    inputs = [input_lang_tokenizer.word_index[i] for i in sentence.split(' ')]
    inputs = keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    result = ''
    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([target_lang_tokenizer.word_index['<start>']], 0)
    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        attention_weights = tf.reshape(attention_weights, (-1, ))
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += target_lang_tokenizer.index_word[predicted_id] + ' '
        if target_lang_tokenizer.index_word[predicted_id] == '<end>':
            return result, sentence
        dec_input = tf.expand_dims([predicted_id], 0)
    return result, sentence

In [49]:
checkpoint.restore(tf.train.latest_checkpoint(''))

In [50]:
evaluate('hace mucho frio aqui.')

('it s very good here . <end> ', '<start> hace mucho frio aqui . <end>')

In [51]:
# MasoudKaviani.ir